# Création de prompts pour le nommage et la description des clusters

In [ ]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds") # nolint
df_pilot1_2022 <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_2022.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_clusters_composition.rds")

In [ ]:
# Identifier les variables qui ont été utilisées pour la construction 
# des clusters à analyser
vars_clusters <- colnames(df_pilot1_2022)[colnames(df_pilot1_2022) %in% colnames(kmeans_result$centers)] # nolint: line_length_linter.

In [ ]:
dfUsedForClustering <- df_pilot1_2022
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

df_prompts <- tibble(cluster_name = character(), prompt = character())

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculs des proportions de femmes, d'immigrants, âge et revenu moyen
  perc_femmes <- mean(cluster_data$female, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$immigrant, na.rm = TRUE) * 100
  age_mean <- mean(cluster_data$age, na.rm = TRUE) * 80
  revenu_moyen <- mean(cluster_data$ses_income, na.rm = TRUE) * 225000

  # Génération du prompt de suggestion de noms
  cat(paste0("Pour chacun des personnas décrit, tu pourrais trouver un nom québécois et donner une brève description du personna.\n",

             "\nPour le persona ", cluster_value, " :\n",
             "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
             if (any(cluster_vars$color == "green")) {
               paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
             } else {
               "Aucune variable supérieure ne se distingue fortement."
             },
             "\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
             if (any(cluster_vars$color == "red")) {
               paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
             } else {
               "Aucune variable inférieure ne se distingue fortement."
             },
             "\n\nCe cluster est composé de ", round(perc_femmes, 1), "% de femmes, ", 
             round(perc_immigrants, 1), "% d'immigrants, un âge moyen de ", round(age_mean, 1), 
             " ans et un revenu moyen de ", round(revenu_moyen, 1), " k$.\n"
  ))

  # df_prompt <- data.frame(
  #   cluster_name = cluster_value, 
  #   prompt = prompt)

  # df_prompts <- bind_rows(df_prompts, df_prompt)
}